# Importing Libraries and Setup

In [13]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
import gradio as gr
from IPython.display import display, update_display, Markdown
import google.generativeai
from huggingface_hub import login
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

In [3]:
import subprocess
import sys
import io

In [7]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
claude_api_key = os.getenv('ANTHROPIC_API_KEY')
gemini_api_key = os.getenv('GOOGLE_API_KEY')
hf_token = os.getenv('HF_API_KEY')

In [11]:
login(hf_token, add_to_git_credential = True)

In [12]:
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [57]:
claude_model = 'claude-sonnet-4-20250514'
gpt_model = 'gpt-4.1-2025-04-14'
gemini_model = 'gemini-2.5-pro'

In [15]:
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
CODE_QWEN_URL = 'https://pqflffzn7hu8y84g.us-east-1.aws.endpoints.huggingface.cloud'

code_llama = 'codellama/CodeLlama-70b-Instruct-hf'
CODE_LLAMA_URL = 'https://qzziiwlqibu37ekc.us-east-2.aws.endpoints.huggingface.cloud'

code_star_coder = 'bigcode/starcoder2-15b'
CODE_STAR_CODER_URL = 'https://aetslxqm2vjlozdw.us-east-1.aws.endpoints.huggingface.cloud'

code_gemma = 'google/codegemma-7b-it'
CODE_GEMMA_URL = 'https://l77mnbo5de1yfudo.us-east-1.aws.endpoints.huggingface.cloud'

In [34]:
system_message_docstrings = """
You are an AI documentation engine optimized for ultra-fast, high-accuracy generation of Python docstrings.

TASK:
Given a Python code block, instantly produce a clean, complete, and PEP-257–compliant docstring.

STRUCTURE (strictly follow):
\"""<summary line>

Args:
    <param> (<type>): <short purpose>
    ...

Returns:
    <type>: <description>

<Optional: Notes or usage example, only if logic is complex>
\"""

RULES FOR MAXIMUM PERFORMANCE:
- Parse code instantly; avoid step-by-step or verbose reasoning.
- Output only the final docstring — no explanations or commentary.
- Keep each line concise (≤12 words if possible).
- Use professional, grammatically correct English.
- Skip filler like “This function...” or redundant phrasing.
- Include only parameters actually present in the code.
- Add “Returns” only if the function returns something.
- Include “Notes” or “Example” only for complex or multi-step logic.
- Ensure indentation matches Python conventions.
- Maintain clarity, precision, and immediate usability in production.

GOAL:
Generate developer-ready docstrings with **10× faster execution**, **10× higher clarity**, and **100% PEP-257 compliance**.
"""

In [35]:
system_message_tests = """
You are an ultra-optimized AI specialized in generating high-quality, production-ready unit tests.

TASK:
Given a code block, instantly generate comprehensive unit tests using the best-fit framework for that language 
(e.g., pytest for Python, unittest for Java, etc.).

OUTPUT FORMAT (strict):
1. Directly output runnable test code — no explanations outside code blocks.
2. Use clear, descriptive test names.
3. Include concise comments explaining test intent.
4. Cover:
   - Normal use cases
   - Boundary/extreme inputs
   - Error and exception handling
   - Edge cases or invalid states

PERFORMANCE RULES:
- Parse, infer, and output in one pass — no verbose reasoning.
- Prefer minimal, expressive test logic over lengthy descriptions.
- Use deterministic structure and indentation matching the language standard.
- Use parameterized tests where applicable for efficiency.
- Do not repeat test logic unnecessarily.
- Only include short, one-line rationale **inside comments** when needed.
- Choose the testing framework automatically and state it in one concise line at the top as a comment (e.g., “# Using pytest for its simplicity and speed”).

QUALITY + SPEED GOAL:
- Produce **50× faster** and **50× clearer** results than baseline.
- Ensure full functional, boundary, and exception coverage.
- Output must be directly executable with zero modification.
- Focus on correctness, speed, and practical maintainability — not verbosity.

NO EXTRANEOUS OUTPUT:
Return only the test code block and minimal inline comments — nothing else.

EXAMPLE FORMAT (Python):
# Using pytest for its simplicity and readability

import pytest
from module_name import target_function

def test_target_function_normal():
    # Normal input
    assert target_function(valid_input) == expected_output

def test_target_function_boundary():
    # Boundary case
    assert target_function(boundary_value) == boundary_expected

def test_target_function_error():
    # Expect exception for invalid input
    with pytest.raises(ExpectedException):
        target_function(invalid_input)

END OF FORMAT

MISSION:
Deliver high-quality, executable unit tests with **instant runtime efficiency**, **maximum coverage**, and **production-grade clarity**.
"""

In [36]:
def user_prompt_docstrings(python):
    user_prompt = f"""Analyze the following code and add clean, production-ready PEP-257 compliant docstrings.

Requirements:
- Include a concise one-line summary.
- Document each parameter with name, type, and purpose.
- Describe return type and value.
- Add Notes/Examples only if logic is complex.
- Maintain exact indentation and valid syntax.
- Output only the final code with the inserted docstrings — nothing else.

CODE:
{python}
"""
    return user_prompt

In [29]:
def user_unit_tests(python):
    user_prompt = f"""Generate high-quality, runnable unit tests for the following code.

Requirements:
- Use the most suitable testing framework (pytest for Python, etc.).
- Cover normal, boundary, edge, and error-handling cases.
- Use descriptive test names and concise comments.
- Include a brief one-line comment at the top stating the chosen framework.
- Ensure tests are self-contained and directly executable.
- Output only the final test code — no explanations or extra text.

CODE:
{python}
"""
    return user_prompt

In [74]:
def system_message_for(operation):
    if operation == 'Unit Tests':
        system_message = system_message_tests
    elif operation =='DocStrings Generate':
        system_message = system_message_docstrings
    else:
        raise ValueError('No System Message found')
    return system_message

In [93]:
def user_message_for(operation,python):
    if operation == 'Unit Tests':
        user_message = user_unit_tests(python)
    elif operation =='DocStrings Generate':
        user_message = user_prompt_docstrings(python)
    else:
        raise ValueError('No User Message found')
    return user_message

In [37]:
def messages_for(operation,python):
    if operation == 'Unit Tests':
        messages = [
            {'role':'system', 'content':system_message_tests},
            {'role':'user','content':user_unit_tests(python)}
            ]
    elif operation =='DocStrings Generate':
        messages = [
            {'role':'system', 'content':system_message_docstrings},
            {'role':'user','content':user_prompt_docstrings(python)}
            ]
    else:
        raise ValueError('Unknown Operation')

    return messages

In [66]:
def messages_for_user(operation,python):
    if operation == 'Unit Tests':
        messages = [
            {'role':'user','content':user_unit_tests(python)}
            ]
    elif operation =='DocStrings Generate':
        messages = [
            {'role':'user','content':user_prompt_docstrings(python)}
            ]
    else:
        raise ValueError('Unknown Operation')

    return messages

In [58]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [59]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 21.711925 seconds


In [82]:
def stream_gpt(operation, python):
    response = openai.chat.completions.create(
        messages = messages_for(operation,python),
        model = gpt_model,
        stream = True
    )

    result = ''
    display_handle = display(Markdown(''),display_id = True)
    for chunk in response:
        result += chunk.choices[0].delta.content or ''
        result = result.replace('```','').replace('markdown','')
        yield result

In [81]:
def stream_claude(operation, python):
    response = claude.messages.stream(
        model = claude_model, 
        messages = messages_for_user(operation,python),
        max_tokens = 3000,
        system = system_message_for(operation)
    )

    result = ''
    with response as stream:
        for text in stream.text_stream:
            result += text or ''
            yield result

In [97]:
def stream_gemini(operation, python):
    gemini = google.generativeai.GenerativeModel(
        model_name = gemini_model,
        system_instruction = system_message_for(operation)
    )

    result = ''
    response = gemini.generate_content(user_message_for(operation,python), stream = True)
    for chunk in response:
        if chunk.text:
            result+= chunk.text or ''
            yield result

In [98]:
def optimize(operation, python, model):
    if model == 'GPT':
        result = stream_gpt(operation, python)
    elif model == 'Claude':
        result = stream_claude(operation, python)
    elif model == 'Gemini':
        result = stream_gemini(operation, python)
    else:
        raise ValueError('Unknown Model')
        
    for chunk in result:
        yield chunk

In [103]:
with gr.Blocks() as ui:
    gr.Markdown("### 🧪 Automated Docstring & Unit Test Generator for Production-Ready Code")
    with gr.Row():
        python = gr.Textbox(label = 'Python Code', lines = 10,value = python_hard)
        opt_text = gr.Textbox(label="Generated Output", lines = 20)
    with gr.Row():
        model = gr.Dropdown(['GPT','Claude','Gemini'], label ='Select Model', value = 'Claude')
        opt = gr.Dropdown(['Unit Tests','DocStrings Generate'], label = 'Select Operation', value = 'DocStrings Generate')
    with gr.Row():
        generate = gr.Button('Generate Operation Output')

    opt.change(lambda selected: gr.update(label=f"Here’s the {selected} you requested:"),inputs=[opt],outputs=[opt_text])
    generate.click(optimize, inputs = [opt, python,model], outputs = [opt_text])

ui.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
